In [56]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from tqdm import tqdm
from sqlalchemy import create_engine
import datetime

import random
import scipy.stats

데이터 셋 형태

In [54]:
# cust_lst: 고객 정보 e.g., Phone_num
# per_lst: 비율 list e.g., [0.3, 0.4, 0.2, 0.1]
def _rnd(df, per_lst):
    cust_lst = list(df["Phone_Number"])
    num_lst = []
    rem_num = len(cust_lst)
    for i in range(len(per_lst)-1):
        tmp_num = round(len(cust_lst)*per_lst[i])
        num_lst.append(tmp_num)
        rem_num -= tmp_num
    num_lst.append(rem_num)
    seg_lst = []
    
    for i in range(len(num_lst)):
        tmp_lst = random.sample(cust_lst, num_lst[i])
#        print (num_lst[i])
        seg_lst.append(tmp_lst)
        cust_lst = list(set(cust_lst)-set(tmp_lst))
    return seg_lst
    

In [7]:
def _intersection(lst_1, lst_2):
    return (list(set(lst_1)&set(lst_2)))

In [50]:
def _check_seg_overlap(seg_lst):
    for i in range(len(seg_lst)):
        for j in range(i+1, len(seg_lst)):
            if len(_intersection(seg_lst[i], seg_lst[j]))>0:
                print ("샘플 중복 발생")
                return True
            else:
                return False

In [10]:
def _df_transform(df):
    df_m = df.groupby("Phone_Number").agg({'Quantity_divide':"sum", 'Sales_divide': "sum",'Orderid':"nunique"  }).reset_index().rename(columns={"Quantity_divide":"quantity", "Sales_divide": "Sales", "Orderid":"order_count"})
    return df_m

In [43]:
def _check_outlier(df, buffer):
    col_name = list(df)[1:]
    for i in range(len(col_name)):
        threshold = np.percentile(df[col_name[i]], float(buffer[i]))
        df = df[df[col_name[i]]<=threshold]
#        print (threshold)
#        print (q1, q3, iqr)
#        print (df)
#        print (df)
    return df

In [94]:
def _check_seg_sim(df, seg):
    check = True
    col_name = list(df)[1:]
    for i in range(len(seg)):
        for j in range(i, len(seg)):
            df_1 = df[df.Phone_Number.isin(seg[i])]
            df_2 = df[df.Phone_Number.isin(seg[j])]
            for k in range(len(col_name)):
                test1 = list(df_1[col_name[k]])
                test2 = list(df_2[col_name[k]])
                if scipy.stats.ttest_ind(test1, test2, equal_var=True).pvalue < 0.3:
                    check = False
#                    print (check)
                if scipy.stats.f_oneway(test1, test2).pvalue < 0.3:
                    check = False 
    return check

In [98]:
df_m = _df_transform(df) 
df_m = _check_outlier(df_m, [99.9, 99.9, 99.9])


In [113]:
fin = False
i = 0
#print (_check_seg_overlap(seg_lst))
#print (_check_seg_sim(df_m, seg_lst))

while (not fin):
    seg_lst = _rnd(df_m, [0.3, 0.3, 0.4])
    if _check_seg_overlap(seg_lst)==False:
        if _check_seg_sim(df_m, seg_lst)==True:
            fin = True

In [156]:
def _write_csv(seg):
    for i in range(len(seg)):
        head = "."
        tmp_seg = seg[i]
        filename = "seg_%s" %str(i+1)
        with open("C:/Users/Echo/Desktop/" + filename + ".csv",'w', newline= '') as file :
            write = csv.writer(file)
            write.writerow(head)
            for i in tmp_seg:
                print (i)
                write.writerow([i])
    return 0

In [146]:
def _main(df, outlier, percentage):
    df_m = _df_transform(df) 
    df_m = _check_outlier(df_m, outlier)
    fin = False
    i = 0

    while (not fin):
        seg_lst = _rnd(df_m, percentage)
        if _check_seg_overlap(seg_lst)==False:
            if _check_seg_sim(df_m, seg_lst)==True:
                fin = True
    _write_csv(seg_lst)
    return seg_lst


In [154]:
_main(df, [0.99, 0.99, 0.99], [0.6, 0.2, 0.1, 0.1])